In [2]:
import numpy as np
import pandas as pd

Question 1
Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

botswana.tsv

О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [3]:
dataTask1 = pd.read_csv("botswana.tsv", sep="\t")
np.unique(dataTask1.religion.values)
#4

array(['catholic', 'other', 'protestant', 'spirit'], dtype=object)

Question 2

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [4]:
dataTask2 = pd.read_csv("botswana.tsv", sep="\t")
dataTask2.dropna().shape
#1834

(1834, 15)

Question 3

В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак x1 на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак
x2={1,0,x1='не применимо',иначе;
заменить "не применимо" в x1 на произвольную константу c, которая среди других значений x1 не встречается.
Теперь, когда мы построим регрессию на оба признака и получим модель вида
y=β0+β1x1+β2x2,
на тех объектах, где x1 было измерено, регрессионное уравнение примет вид
y=β0+β1x,
а там, где x1 было "не применимо", получится
y=β0+β1c+β2.
Выбор c влияет только на значение и интерпретацию β2, но не β1.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.
Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.
Замените NaN в признаке agefm на cagefm=0.
У объектов, где nevermarr = 1, замените NaN в признаке heduc на cheduc1=−1 (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
Сколько осталось пропущенных значений в признаке heduc?

In [5]:
dataTask3 = pd.read_csv("botswana.tsv", sep="\t")
#dataTask3['nevermarr'] = np.where(pd.isnull(dataTask3.agefm), 1, 0)
dataTask3['nevermarr'] = dataTask3.agefm.apply(lambda x: 1 if pd.isnull(x) else 0)
dataTask3.drop(["evermarr"], axis=1, inplace=True)
#dataTask3['agefm'] = np.where(pd.isnull(dataTask3.agefm), 0, dataTask3.agefm)
dataTask3['agefm'].fillna(0, inplace=True)
#dataTask3['heduc'] = np.where(dataTask3.nevermarr == 1, -1, dataTask3.heduc)
dataTask3.loc[dataTask3.nevermarr == 1,'heduc']=-1

dataTask3[pd.isnull(dataTask3.heduc)].heduc.shape
#123

(123L,)

Question 4


Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения (cidlnchld=−1, cheduc2=−2 (значение -1 мы уже использовали), cusemeth=−1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [6]:
#dataTask3['idlnchld_noans'] = np.where(pd.isnull(dataTask3.idlnchld), 1, 0)
dataTask3['idlnchld_noans'] = dataTask3.idlnchld.apply(lambda x: 1 if pd.isnull(x) else 0)
#print np.unique(dataTask3.idlnchld_noans.values)
#dataTask3["idlnchld"] = np.where(dataTask3.idlnchld_noans == 1, -1, dataTask3.idlnchld)
dataTask3["idlnchld"].fillna(-1, inplace=True)
#print np.unique(dataTask3.idlnchld_noans.values)

#dataTask3['heduc_noans'] = np.where(pd.isnull(dataTask3.heduc), 1, 0)
dataTask3['heduc_noans'] = dataTask3.heduc.apply(lambda x: 1 if pd.isnull(x) else 0)
#print np.unique(dataTask3.heduc_noans.values)
#dataTask3["heduc"] = np.where(dataTask3.heduc_noans == 1, -2, dataTask3.heduc)
dataTask3["heduc"].fillna(-2, inplace=True)
#print np.unique(dataTask3.heduc_noans.values)

#dataTask3['usemeth_noans'] = np.where(pd.isnull(dataTask3.usemeth), 1, 0)
dataTask3['usemeth_noans'] = dataTask3.usemeth.apply(lambda x: 1 if pd.isnull(x) else 0)
#print np.unique(dataTask3.usemeth_noans.values)
#dataTask3["usemeth"] = np.where(dataTask3.usemeth_noans == 1, -1, dataTask3.usemeth)
dataTask3["usemeth"].fillna(-1, inplace=True)
#print np.unique(dataTask3.usemeth_noans.values)

#print np.unique(dataTask3.agefm.values)

#78264


In [7]:
dataTask3.dropna(inplace=True)
print np.unique(dataTask3.idlnchld_noans.values)
print np.unique(dataTask3.heduc_noans.values)
print np.unique(dataTask3.usemeth_noans.values)
dataTask3.head(5)
print dataTask3.shape, dataTask3.shape[0] * dataTask3.shape[1]


[0 1]
[0 1]
[0 1]
(4348, 18) 78264


Question 5


Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R2? Округлите до трёх знаков после десятичной точки.

Если код из примера у вас не воспроизводится:

убедитесь, что вы сделали так:

import statsmodels.formula.api as smf
возможно, вам нужно обновить библиотеку patsy; выполните в командной строке

pip install -U patsy


In [8]:
dataTask3.head(5)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [9]:
import statsmodels.formula.api as smf
#age + educ + religion + idlnchld + knowmeth +usemeth	evermarr	agefm	heduc	urban	electric	radio	tv	bicycle	nevermarr	idlnchld_noans	heduc_noans	usemeth_noans
m1 = smf.ols("ceb ~ " + ' + '.join(dataTask3.columns[1:]), 
             data=dataTask3)
fitted = m1.fit()
print fitted.summary()
#R-squared:0.644

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:50:58   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Question 6


Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

In [10]:
#religion[T.other] religion[T.protestant] religion[T.spirit]
#3

Question 7


Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.


Ошибка гомоскедастична, p>0.05

Ошибка гетероскедастична, p≤0.05, нужно делать поправку Уайта

In [11]:
import statsmodels.stats.api as sms
print 'Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1]

Breusch-Pagan test: p=0.000000


C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [12]:
m2 = smf.ols("ceb ~ " + ' + '.join(dataTask3.columns[1:]), 
             data=dataTask3)
fitted2 = m2.fit(cov_type='HC1')
print fitted2.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        21:51:03   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Question 8


Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [16]:
#dataTask3.drop(["religion", "radio", "tv"], axis=1, inplace=True)
print "ceb ~ " + ' + '.join(dataTask3.columns[1:])
dataTask3.to_csv("native.csv")

ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans


In [19]:
m4 = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans", 
             data=dataTask3)
fitted4 = m4.fit()

m5 = smf.ols("ceb ~ age + educ + idlnchld + knowmeth + bicycle + usemeth + agefm + heduc + urban + electric + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans", 
             data=dataTask3)
fitted5 = m5.fit()


print fitted4.compare_f_test(fitted5)

(0.91923577846316695, 0.46723055472743502, 5.0)


Question 9


Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением cusemeth=−1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5×10−8, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [14]:
m6 = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans", 
             data=dataTask3)
fitted6 = m6.fit(cov_type='HC1')

m7 = smf.ols("ceb ~ age + educ + religion + idlnchld + knowmeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans", 
             data=dataTask3)
fitted7 = m7.fit(cov_type='HC1')
#print 'Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted5.resid, fitted5.model.exog)[1]
#print fitted5.summary()

print  fitted6.compare_f_test(fitted7)
#p = 1.36341009014e-39

C:\Users\KONYAYEV.MAKSIM\AppData\Local\Continuum\anaconda2\lib\site-packages\statsmodels\regression\linear_model.py:1746: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


(91.36702105075571, 1.363410090136652e-39, 2.0)


Question 10


Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.


Итоговая модель объясняет 63% вариации отклика


У женщин, не знающих, какое количество детей идеально, в среднем на
βidlnchld_noans+cidlnchldβidlnchld≈0.58
детей больше


У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])


У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])


У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])


С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])

In [15]:
print fitted6.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Tue, 20 Mar 2018   Prob (F-statistic):               0.00
Time:                        13:31:17   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1